In [92]:
%reset

import open3d as o3d

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [93]:
test_path = r"C:\Users\pojzi\Programovanie\bakalarka\impact_recognition\data\neutral_images_3d\CFD-AF-200-228-N\CFD-AF-200-228-N.obj"
mesh = o3d.io.read_triangle_mesh(test_path)
point_cloud = mesh.sample_points_uniformly(number_of_points=1000)
point_cloud


PointCloud with 1000 points.

In [94]:
o3d.visualization.draw_geometries([point_cloud])


In [95]:
import numpy as np

In [96]:
array = np.asarray(point_cloud.points)

In [98]:
import os
import open3d as o3d
import numpy as np

class obj2pcl: 
    @staticmethod
    def convert(path, num_points=10000, ouput_name=None):
        if not os.path.exists(path):
            raise FileNotFoundError("The path does to the source .obj file does not exist")
        
        mesh = o3d.io.read_triangle_mesh(path)
        point_cloud = mesh.sample_points_uniformly(number_of_points=num_points)
        
        return point_cloud, np.asarray(point_cloud.points)

In [99]:
import os
os.listdir("../../")

['.ipynb_checkpoints',
 '3DFacePointCloudNet',
 'code',
 'DECA',
 'images',
 'impact_recognition',
 'papers_bakalarka',
 'pointnet2',
 'Pointnet2_PyTorch',
 'PointNeXt',
 'test',
 'xd.txt']

In [116]:
import __init__
import os, argparse, yaml, numpy as np
import sys
sys.path.append("C:\\Users\\pojzi\\Programovanie\\bakalarka\\PointNeXt\\")
                
from torch import multiprocessing as mp
from examples.classification.train import main as train
from examples.classification.pretrain import main as pretrain
from openpoints.utils import EasyConfig, dist_utils, find_free_port, generate_exp_directory, resume_exp_directory, Wandb
import sys


parser = argparse.ArgumentParser('S3DIS scene segmentation training')
# parser.add_argument('--cfg', type=str, required=True, help='config file')
# parser.add_argument('--profile', action='store_true', default=False, help='set to True to profile speed')
args, opts = parser.parse_known_args()

args.cfg = r"C:/Users/pojzi/Programovanie/bakalarka/PointNeXt/cfgs\modelnet40ply2048" + r"/pointnext-s.yaml"
args.profile = False

cfg = EasyConfig()

cfg.load(args.cfg, recursive=True)
cfg.update(opts)
if cfg.seed is None:
    cfg.seed = np.random.randint(1, 10000)

# init distributed env first, since logger depends on the dist info.
cfg.rank, cfg.world_size, cfg.distributed, cfg.mp = dist_utils.get_dist_info(cfg)
cfg.sync_bn = cfg.world_size > 1

# init log dir
cfg.task_name = args.cfg.split('.')[-2].split('/')[-2]
cfg.exp_name = args.cfg.split('.')[-2].split('/')[-1]
tags = [
    cfg.task_name,  # task name (the folder of name under ./cfgs
    cfg.mode,
    cfg.exp_name,  # cfg file name
    f'ngpus{cfg.world_size}',
    f'seed{cfg.seed}',
]
opt_list = [] # for checking experiment configs from logging file
for i, opt in enumerate(opts):
    if 'rank' not in opt and 'dir' not in opt and 'root' not in opt and 'pretrain' not in opt and 'path' not in opt and 'wandb' not in opt and '/' not in opt:
        opt_list.append(opt)
cfg.root_dir = os.path.join(cfg.root_dir, cfg.task_name)
cfg.opts = '-'.join(opt_list)

# added here 
# cfg.mode = 'test'
if cfg.mode in ['resume', 'val', 'test']:
    resume_exp_directory(cfg, pretrained_path=cfg.pretrained_path)
    cfg.wandb.tags = [cfg.mode]
else:  # resume from the existing ckpt and reuse the folder.
    generate_exp_directory(cfg, tags, additional_id=os.environ.get('MASTER_PORT', None))
    cfg.wandb.tags = tags
os.environ["JOB_LOG_DIR"] = cfg.log_dir
cfg_path = os.path.join(cfg.run_dir, "cfg.yaml")
with open(cfg_path, 'w') as f:
    yaml.dump(cfg, f, indent=2)
    os.system('copy {} {}'.format(args.cfg, cfg.run_dir))

cfg.cfg_path = cfg_path
cfg.wandb.name = cfg.run_name

if cfg.mode == 'pretrain':
    main = pretrain
else:
    main = train

# multi processing.
if cfg.mp:
    port = find_free_port()
    cfg.dist_url = f"tcp://localhost:{port}"
    print('using mp spawn for distributed training')
    mp.spawn(main, nprocs=cfg.world_size, args=(cfg, args.profile))
else:
    cfg.log_path = os.path.join(os.getcwd(), cfg.log_path).replace("\\", "/")
    
    model = main(0, cfg, profile=args.profile)

launch mp with 1 GPUs, current rank: 0


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/pojzi/Programovanie/bakalarka/impact_recognition/data/log/cfgs/modelnet40ply2048/cfgs/modelnet40ply2048-train-pointnext-s-ngpus1-seed4489-20240417-153125-mELZWxbwHoF4r52TKVQmer/cfgs/modelnet40ply2048-train-pointnext-s-ngpus1-seed4489-20240417-153125-mELZWxbwHoF4r52TKVQmer.log'

In [106]:
args.cfg.split('.')

['C:\\Users\\pojzi\\Programovanie\\bakalarka\\PointNeXt\\cfgs\\modelnet40ply2048\\pointnext-s',
 'yaml']

In [88]:
log_path = r"C:\Users\pojzi\Programovanie\bakalarka\PointNeXt\examples\classification\log\modelnet40ply2048\modelnet40ply2048-train-pointnext-s-ngpus1-seed350-20240417-104017-X2664SgbpVBX4cd5W3vCPm"
log_file = list(filter(lambda x: x.endswith("log"), os.listdir(log_path)))[0]

full_path = os.path.join(log_path, log_file)
print(full_path)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\pojzi\\Programovanie\\bakalarka\\PointNeXt\\examples\\classification\\log\\modelnet40ply2048\\modelnet40ply2048-train-pointnext-s-ngpus1-seed350-20240417-104017-X2664SgbpVBX4cd5W3vCPm'

In [79]:
os.path.exists("C:/Users/pojzi/Programovanie/bakalarka/PointNeXt/examples/classification/log/modelnet40ply2048/modelnet40ply2048-train-pointnext-s-ngpus1-seed4240-20240417-150141-bZZkVVuJsoydXWMmaxnoVc/last_log.log")

False

In [68]:
first_file = os.listdir(log_path)[-1]
first_file
full_path = os.path.join(log_path, first_file)

os.listdir(log_path)


file = open(full_path)
data = file.read()

print(data)
file.close()

[04/17 10:40:17] ModelNet40Ply2048 INFO: dist_url: tcp://localhost:8888
dist_backend: nccl
multiprocessing_distributed: False
ngpus_per_node: 1
world_size: 1
launcher: mp
local_rank: 0
use_gpu: True
seed: 350
epoch: 0
epochs: 600
ignore_index: None
val_fn: validate
deterministic: False
sync_bn: False
criterion_args:
  NAME: SmoothCrossEntropy
  label_smoothing: 0.2
use_mask: False
grad_norm_clip: 1
layer_decay: 0
step_per_update: 1
start_epoch: 1
sched_on_epoch: True
wandb:
  use_wandb: False
  project: PointNeXt-ModelNet40Ply2048
  tags: ['modelnet40ply2048', 'train', 'pointnext-s', 'ngpus1', 'seed350']
  name: modelnet40ply2048-train-pointnext-s-ngpus1-seed350-20240417-104017-X2664SgbpVBX4cd5W3vCPm
use_amp: False
use_voting: False
val_freq: 1
resume: False
test: False
finetune: False
mode: train
logname: None
load_path: None
print_freq: 10
save_freq: -1
root_dir: log/modelnet40ply2048
pretrained_path: None
datatransforms:
  train: ['PointsToTensor', 'PointCloudScaleAndTranslate']
  v

In [111]:
import os
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, path_to_data, num_samples=1):
        if not os.path.exists(path_to_data):
            raise FileNotFoundError("The path does not exist")
        
        self.num_samples = num_samples
        self.datadir = path_to_data
        self.absolute_path = os.path.join(os.getcwd(), self.datadir)
        self.labels = self.find_classes()
        self.data = self.create_data()
        
        print("Found {} classes".format(self.__len__()))
        
        # Create class-to-index mapping
        self.class_to_idx = {class_: idx for idx, class_ in enumerate(self.labels)}
            
    def find_classes(self):
        directories = os.listdir(self.datadir)
        return [x for x in directories if os.path.isdir(os.path.join(self.absolute_path, x))][:self.num_samples]
        
    def create_data(self):
        data = []
        for class_ in self.labels:
            obj_path = os.path.join(self.absolute_path, class_, class_ + '.obj')
            _, pc_array = obj2pcl.convert(obj_path)
            data.append(pc_array)
        return data
            
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index], self.class_to_idx[self.labels[index]]

dest = os.path.join("neutral_images_3d")

# Create an instance of CustomDataset
dataset = CustomDataset(dest, num_samples=5)

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Print the class-to-index mapping
print("Class to Index Mapping:", dataset.class_to_idx)

Found 5 classes
Class to Index Mapping: {'CFD-AF-200-228-N': 0, 'CFD-AF-201-060-N': 1, 'CFD-AF-202-122-N': 2, 'CFD-AF-203-077-N': 3, 'CFD-AF-204-067-N': 4}


In [21]:
os.listdir("neutral_images_3d")[0]

'CFD-AF-200-228-N'

In [33]:
absolute_path = os.path.join(os.getcwd(), "neutral_images_3d")

os.path.isdir(os.path.join(absolute_path, os.listdir("neutral_images_3d")[0]))

True

In [23]:
os.path.exists('CFD-AF-200-228-N')

False

In [113]:
inpt, label = next(iter(dataloader))

In [119]:
label

tensor([0, 4])